# Исследование надёжности заёмщиков

Наш клиент - кредитный отдел банка. Необходимо разобраться, влияет ли семейное положение клиента и количество детей на факт своевременного погашения кредита. Нам предоставлена статистика платежеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — системы, оценивающей способность потенциального заёмщика вернуть кредит банку.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Обзор-данных-" data-toc-modified-id="Обзор-данных--1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Обзор данных <a id="start"></a></a></span></li><li><span><a href="#Предобработка-данных-" data-toc-modified-id="Предобработка-данных--2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Предобработка данных <a id="preprocessing"></a></a></span><ul class="toc-item"><li><span><a href="#Обработка-пропусков-" data-toc-modified-id="Обработка-пропусков--2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Обработка пропусков <a id="filling"></a></a></span></li></ul></li><ul class="toc-item"><li><span><a href="#Замена-типа-данных-" data-toc-modified-id="Замена-типа-данных--12.1"><span class="toc-item-num">12.1&nbsp;&nbsp;</span>Замена типа данных <a id="typing"></a></a></span></li><li><span><a href="#Обработка-дубликатов-" data-toc-modified-id="Обработка-дубликатов--12.2"><span class="toc-item-num">12.2&nbsp;&nbsp;</span>Обработка дубликатов <a id="dupes"></a></a></span></li></ul></li><ul class="toc-item"><li><span><a href="#Лемматизация-" data-toc-modified-id="Лемматизация--14.1"><span class="toc-item-num">14.1&nbsp;&nbsp;</span>Лемматизация <a id="lemmatizing"></a></a></span></li></ul></li><ul class="toc-item"><li><span><a href="#Категоризация-данных-" data-toc-modified-id="Категоризация-данных--16.1"><span class="toc-item-num">16.1&nbsp;&nbsp;</span>Категоризация данных <a id="binning"></a></a></span></li></ul></li><li><span><a href="#Вопросы-исследования-" data-toc-modified-id="Вопросы-исследования--21"><span class="toc-item-num">21&nbsp;&nbsp;</span>Вопросы исследования <a id="analysis"></a></a></span><ul class="toc-item"><li><span><a href="#Есть-ли-зависимость-между-наличием-детей-и-возвратом-кредита-в-срок?-" data-toc-modified-id="Есть-ли-зависимость-между-наличием-детей-и-возвратом-кредита-в-срок?--21.1"><span class="toc-item-num">21.1&nbsp;&nbsp;</span>Есть ли зависимость между наличием детей и возвратом кредита в срок? <a id="children"></a></a></span></li><li><span><a href="#Есть-ли-зависимость-между-семейным-положением-и-возвратом-кредита-в-срок?-" data-toc-modified-id="Есть-ли-зависимость-между-семейным-положением-и-возвратом-кредита-в-срок?--21.2"><span class="toc-item-num">21.2&nbsp;&nbsp;</span>Есть ли зависимость между семейным положением и возвратом кредита в срок? <a id="fam_status"></a></a></span></li></ul></li><ul class="toc-item"><li><span><a href="#Есть-ли-зависимость-между-уровнем-дохода-и-возвратом-кредита-в-срок?-" data-toc-modified-id="Есть-ли-зависимость-между-уровнем-дохода-и-возвратом-кредита-в-срок?--23.1"><span class="toc-item-num">23.1&nbsp;&nbsp;</span>Есть ли зависимость между уровнем дохода и возвратом кредита в срок? <a id="income"></a></a></span></li><li><span><a href="#Как-разные-цели-кредита-влияют-на-его-возврат-в-срок?-" data-toc-modified-id="Как-разные-цели-кредита-влияют-на-его-возврат-в-срок?--23.2"><span class="toc-item-num">23.2&nbsp;&nbsp;</span>Как разные цели кредита влияют на его возврат в срок? <a id="purpose"></a></a></span></li></ul></li><li><span><a href="#Общий-вывод-" data-toc-modified-id="Общий-вывод--24"><span class="toc-item-num">24&nbsp;&nbsp;</span>Общий вывод <a id="conclusion"></a></a></span></li>

## Обзор данных <a id="start"></a>

Выведем общую информацию и начало таблицы:

In [1]:
%pip install pymystem3

In [2]:
import pandas as pd
from pymystem3 import Mystem

In [3]:
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Вывод**

Предоставлены данные о 21525 клиентах по 12 признакам (в т.ч. 4 количественным и 8 категориям). Заголовки колонок удовлетворительны.

Сразу налицо недостающие данные в колонках `days_employed` (трудовой стаж) и `total_income` (ежемесячный доход); ниже мы проверим предположение, что в строке есть либо оба показателя, либо ни одного.

Столбец `days_employed` содержит отрицательные числа, кроме того, его целесообразно привести к целочисленному типу.

Количество знаков после запятой в столбце `total_income` избыточно. Для простоты его также можно привести к целочисленному типу.

Данные об образовании (столбец `education`) записаны то со строчной буквы, то с прописной, желательно приведение к одному регистру.

## Предобработка данных <a id="preprocessing"></a>

### Обработка пропусков <a id="filling"></a>

Прежде всего: действительно ли отсутствие данных о доходе и стаже связяно?

In [5]:
print ('Строк, где отсутствует лишь один из показателей:', len (data[(~data['days_employed'].isnull()) & (data['total_income'].isnull())]))

Строк, где отсутствует лишь один из показателей: 0


Данные в колонках `total_income` и `days_employed` отстутствуют всегда попарно, как мы и предполагали.

Проверим категории видов занятости:

In [6]:
data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Типы занятости выглядят корректно. Представляется разумным сгруппировать данные о доходе и стаже по видам занятости, поскольку диапазон дохода, к примеру, бизнесменов и пенсионеров заметно различается:

In [7]:
# Группируем таблицу по видам занятости:
income_type_grouping = data.groupby('income_type')

# Вычислим медианные значения дохода по видам занятости:
income_type_grouping_median = income_type_grouping['total_income'].median()
print ('Медианный месячный доход:')
income_type_grouping_median

Медианный месячный доход:


income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

In [8]:
# Вычислим средние значения стажа по видам занятости:
print ('Средний стаж:')
income_type_grouping['days_employed'].mean()

Средний стаж:


income_type
безработный        366413.652744
в декрете           -3296.759962
госслужащий         -3399.896902
компаньон           -2111.524398
пенсионер          365003.491245
предприниматель      -520.848083
сотрудник           -2326.499216
студент              -578.751554
Name: days_employed, dtype: float64

Заполнить пропуски в столбце `total_income` медианными значениями по виду занятости выглядит целесообразным.

А вот значения трудового стажа выглядят очень странно: у безработных и пенсионеров стаж нереально велик (в среднем 100 лет и более), а у прочих величины отрицательные.
Попробуем трактовать отрицательные величины как ошибочно записанные положительные:

In [9]:
# Применим стандарную фунцию модуля к значениям стажа:
data['days_employed'] = data['days_employed'].apply(abs)

# Вычислим средние значения по видам занятости:
income_type_grouping_mean_days = income_type_grouping['days_employed'].mean()
print ('Средний стаж (скорректированный):')
income_type_grouping_mean_days

Средний стаж (скорректированный):


income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          3399.896902
компаньон            2111.524398
пенсионер          365003.491245
предприниматель       520.848083
сотрудник            2326.499216
студент               578.751554
Name: days_employed, dtype: float64

Категории безработных и пенсионеров по-прежнему выглядят аномально, но остальные относительно правдоподобно.

Заполним недостающие значения стажа средним по виду занятости.

In [10]:
for income_type in data['income_type'].unique():
    data.loc[(data['days_employed'].isnull()) & (data['income_type'] == income_type), 'days_employed'] = income_type_grouping_mean_days[income_type]

Для заполнения данных о доходах попробуем применить более глубокий анализ. Медианный доход может различаться еще и в зависимости от образования и возраста, проверим эти предположения:

In [11]:
# Группируем таблицу по уровню образования:
income_type_grouping_by_edu = data.groupby('education_id')
# И сразу же проверим качество данных в столбцах education и education_id и их соответствие друг другу:
income_type_grouping_by_edu['education'].value_counts()

education_id  education          
0             высшее                  4718
              ВЫСШЕЕ                   274
              Высшее                   268
1             среднее                13750
              СРЕДНЕЕ                  772
              Среднее                  711
2             неоконченное высшее      668
              Неоконченное высшее       47
              НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
3             начальное                250
              НАЧАЛЬНОЕ                 17
              Начальное                 15
4             ученая степень             4
              УЧЕНАЯ СТЕПЕНЬ             1
              Ученая степень             1
Name: education, dtype: int64

Колонка `education` содержит дублированные категории, отличающиеся только регистром, но соответствующие значения `education_id` корректны, ими можно пользоваться для группировки:

In [12]:
# Вычислим медианные значения дохода по видам занятости:
income_type_grouping_by_edu_median = income_type_grouping_by_edu['total_income'].median()
print ('Медианный месячный доход по уровню образования:')
income_type_grouping_by_edu_median.sort_values(ascending=False)

Медианный месячный доход по уровню образования:


education_id
0    175340.818855
2    160115.398644
4    157259.898555
1    136478.643244
3    117137.352825
Name: total_income, dtype: float64

Действительно, корреляция налицо, выпадают лишь клиенты с ученой степенью (на то могут быть причины, например многие из них работают в сфере образования с невысокими зарплатами), но их количество слишком мало для репрезентативной выборки.

In [13]:
# Проверим, много ли пропущено в этой категории:
data[data['education_id']==4]['total_income'].isnull().sum()

0

Но у этой категории нет пропусков в данных о доходе, поэтому для нас это не проблема.

Теперь проверим корреляцию доходов с возрастом. Прежде всего, проверим данные о возрасте на предмет аномалий:

In [14]:
print ('Нулевых записей о возрасте:', len(data[data['dob_years']==0]))
print ('Некорректных записей о возрасте:', len(data[data['dob_years']<0]))
print ('Сомнительных записей о возрасте:', len(data[(data['dob_years']>100)|((data['dob_years']<18)&(data['dob_years']>0))]))

Нулевых записей о возрасте: 101
Некорректных записей о возрасте: 0
Сомнительных записей о возрасте: 0


Есть небольшое количество нулевых значений возраста, но никаких отрицательных или неправдоподобных величин (младше 18 или старше 100 лет).
Посмотрим, насколько оптимально использовать медиану для заполнения отсутствующих данных:

In [15]:
income_type_grouping_median_age = income_type_grouping['dob_years'].median()
print ('Медианный возраст по видам занятости:')
income_type_grouping_median_age

Медианный возраст по видам занятости:


income_type
безработный        38.0
в декрете          39.0
госслужащий        40.0
компаньон          39.0
пенсионер          60.0
предприниматель    42.5
сотрудник          39.0
студент            22.0
Name: dob_years, dtype: float64

Как и следовало ожидать, категории пенсионеров и студентов имеют свою медиану, остальные достаточно близки. Заполним нулевые значения:

In [16]:
for income_type in data['income_type'].unique():
    data.loc[(data['dob_years']==0) & (data['income_type'] == income_type), 'dob_years'] = income_type_grouping_median_age[income_type]
print ('Нулевых записей о возрасте:', len(data[data['dob_years']==0]))

Нулевых записей о возрасте: 0


Теперь мы можем выделить возрастные категории и оценить целесообразность группировки данных о доходе по ним.

Взглянем на распределение значений в колонке `dob_years`:

In [17]:
data['dob_years'].describe().apply("{:.0f}".format)

count    21525
mean        43
std         12
min         19
25%         34
50%         43
75%         53
max         75
Name: dob_years, dtype: object

В целом разумным выглядит разбиение как на возрастные категории равного размера, так и по декадам. Применим второй подход для наглядности:

In [18]:
# Выделим возрастные группы по условным границам: до 29 лет, 30-39 лет, 40-49 лет и т. д.
data['age_group'] = pd.cut (data['dob_years'], bins=[18, 30, 40, 50, 60, 100], right=False,
                            labels=['до 29','30-39','40-49','50-59','60+'])
data['age_group'].value_counts()

30-39    5749
40-49    5377
50-59    4679
до 29    3183
60+      2537
Name: age_group, dtype: int64

Оценим полезность группировки.

In [19]:
# Группируем таблицу:
income_type_grouping_by_age = data.groupby('age_group')
# Вычислим медианные значения дохода по возрастным группам:
income_type_grouping_by_age_median = income_type_grouping_by_age['total_income'].median()
print ('Медианный месячный доход по возрастным группам:')
income_type_grouping_by_age_median

Медианный месячный доход по возрастным группам:


age_group
до 29    142141.582776
30-39    154416.954788
40-49    154744.914039
50-59    138769.212810
60+      123419.911371
Name: total_income, dtype: float64

Разница есть: молодежи без опыта и людям старше 50 труднее найти высокооплачиваемую работу, а кто-то уже и на пенсии.

Сведем наши группировки:

In [20]:
# Сгруппируем таблицу сразу по трем признакам:
income_type_grouping_multiple = data.groupby(['income_type', 'age_group', 'education_id'])

Заполним пустые значения медианой:

In [21]:
data['total_income'] = data['total_income'].fillna(income_type_grouping_multiple['total_income'].transform('median'))
data.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        2
purpose             0
age_group           0
dtype: int64

Группировка оказалась довольно узкой: две строки, по-видимому, имеют уникальную комбинацию групп и не имеют сведений о доходах, так что две подгруппы остались без медиан. Брать группировку по меньшему числу столбцов только для них несколько непоследовательно, но эти строки содержат и другие полезные для исследования данные, поэтому их предпочтительней сохранить.

In [22]:
# Используем уже вычисленные медианы по видам занятости:
data['total_income'] = data['total_income'].fillna(income_type_grouping['total_income'].transform('median'))   
data.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
age_group           0
dtype: int64

Пропусков больше нет, изучим на предмет аномалий другие данные.

Проверим данные о количестве детей:

In [23]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Здесь наблюдаются аномальные величины -1 и 20. Первое будем трактовать как ошибочно распознанное 1; второе, вероятнее всего, искаженное 2. Сделаем соответствующую замену:

In [24]:
data['children'] = data['children'].apply(abs)
data['children'] = data['children'].replace(20, 2)
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Проверим оставшиеся категории:

In [25]:
print ('Семейное положение:')
print (data['family_status'].value_counts())
print (data['family_status_id'].value_counts())
print()
print ('Пол:')
print (data['gender'].value_counts())
print()
print ('Наличие задолженности:')
print (data['debt'].value_counts())

Семейное положение:
женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64
0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

Пол:
F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Наличие задолженности:
0    19784
1     1741
Name: debt, dtype: int64


В столбце `gender` (пол) присутствует одна ячейка со значением 'XNA' (вероятно, искаженное 'N/A'). Хотя это и уникальное значение, нет оснований считать его некорректным - пол может быть и не указан.

Дубликаты в столбце `education` будут обработаны на следующем этапе.

Остальные значения выглядят корректно.

**Вывод**

Можно заключить, что существенное количество клиентов (порядка 10%) не предоставили данные о своем доходе. Эти пропуски мы заполнили медианными значениями на основании типа занятости, а также образования и возраста клиента, когда это было возможным.

У небольшого числа клиентов не указан возраст (в таблице стоит нулевое значение). Эти пропуски мы заполнили медианными значениями по типу занятости.

Присутствуют также аномалии в записях о детях: отрицательные числа (вероятно, ошибка при распознавании рукописного текста) и часто встречающееся аномальное число 20. Отрицательные значения мы трактуем как положительные, а 20 как ошибочно записанное 2. (Здесь был бы желателен комментарий от выгружавшего данные, есть вероятность, что аномальное значение имеет особое толкование.)

В данных о трудовом стаже аномальные значения преобладают. Надежная трактовка этих данных не представляется возможной без дополнительной информации от авторов выгрузки. Недостающие данные были заполнены средними значениями по виду занятости, но для более качественных данных может быть целесообразным заполнение соответствующими медианными значениями по возрастным категориям.

Виды занятости "предприниматель", "безработный", "студент" и "в декрете" представлены 1-2 строками, и репрезентативность их сомнительна.

Тем не менее, объем и качество данных для решения поставленной задачи (семейное положение, количество детей, сумма дохода и наличие задолженности) можно признать удовлетворительным.

### Замена типа данных <a id="typing"></a>

Заменим тип данных в колонках `days_employed` и `total_income` на целочисленный:

In [26]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.dtypes

children               int64
days_employed          int32
dob_years            float64
education             object
education_id           int64
family_status         object
family_status_id       int64
gender                object
income_type           object
debt                   int64
total_income           int32
purpose               object
age_group           category
dtype: object

Вывод *data.dtypes* говорит о том, что данные в столбце `dob_years` (возраст) также имеют вещественный тип (*float64*), хотя используются как целые числа. Исправим и это:

In [27]:
data['dob_years'] = data['dob_years'].astype('int')
data.dtypes

children               int64
days_employed          int32
dob_years              int32
education             object
education_id           int64
family_status         object
family_status_id       int64
gender                object
income_type           object
debt                   int64
total_income           int32
purpose               object
age_group           category
dtype: object

**Вывод**

Данные в колонках `dob_years`, `days_employed` и `total_income` мы привели к целочисленному типу с помощью *astype()* (*to_numeric()* в данном случае неуместен, т.к. предназначен для приведения к вещественному типу).

### Обработка дубликатов <a id="dupes"></a>

Проверим таблицу на очевидные дубликаты (идентичные строки):

In [28]:
data.duplicated().sum()

54

Исправим обнаруженные ранее дублированные категории в столбце `education` приведением к одному регистру:

In [29]:
data['education'] = data['education'].str.lower()
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Перепроверям количество дублированных строк:

In [30]:
data.duplicated().sum()

71

Число идентичных строк, как видим, выросло. Так незначительный, казалось бы, недочет в заполнении столбца `education` приводит к лишним тратам рабочего времени.

In [31]:
# Удаление идентичных строк:
data = data.drop_duplicates().reset_index(drop=True)
# Проверим результат:
data.duplicated().sum()

0

**Вывод**

Обнаружена 71 дублированная строка, удаленная с помощью *drop_duplicates()*. Скорее всего имеет место комбинация технического и человеческого факторов, так как часть строк отличается только регистром в одном из столбцов.

Дублированные категории в столбце `education` исправлены приведением к нижнему регистру. Очевидно, данные столбца заполняются вручную, выбор из списка был бы более оптимальным.

### Лемматизация <a id="lemmatizing"></a>

Оценим наиболее популярные причины кредитования:

In [32]:
data['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

По сути, выделяется всего 4 ключевых понятия: свадьба, недвижимость/жилье, авто, образование.

Выделим соответствующие категории отдельным столбцом:

In [33]:
m = Mystem()

categories = ['свадьба', 'недвижимость', 'жилье', 'автомобиль', 'образование']

# Лемматизируем данные из строки purpose и проверяем на наличие ключевого слова:
def categorize (purpose):
    try:
        for word in m.lemmatize(purpose):
            if word in categories:
                return word
    except:
        return '<данные повреждены>'
    return 'другое'

# Добавляем новый стобец purpose_group с нашей категоризацией:
data['purpose_group'] = data['purpose'].apply(categorize)
# Категорию 'жилье' включаем в категорию 'нежвижимость':
data['purpose_group'] = data['purpose_group'].replace('жилье', 'недвижимость')
# Проверяем результат:
data['purpose_group'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_group, dtype: int64

Неохваченных строк по итогу категоризации не остается.

**Вывод**

Лемматизация причин взятия кредита (колонка `purpose`) и проверка на соответсвие одному из пяти ключевых слов ('свадьба', 'недвижимость', 'жилье', 'автомобиль', 'образование'), два из которых эквивалентны по смыслу, позволили свести все причины взятия кредита к 4 основным категориям.

Категории добавлены в таблицу отдельной колонкой `purpose_group`.

### Категоризация данных <a id="binning"></a>

Помимо уже добавленных категорий `purpose_group` (категоризация по основным целям получения кредита) <font color='purple'>и `age_group` (возрастные группы)</font>, для цели исследования важны семейное положение и количество детей клиента. Первое уже категоризовано достаточно хорошо в колонке `family_status`. Второе (колонка `children`) также можно считать категоризованным (число уникальных значений мало), но разумно будет ввести и дополнительную бинарную категорию: наличие/отстутствие детей у клиента:

In [34]:
def has_children (num):
    try:
        if num > 0:
            return 1
        else:
            return 0
    except:
        print ('На входе не численные данные!')
    
# Результат категоризации внесем в отдельный столбец has_children:
data['has_children'] = data['children'].apply(has_children)

Еще одной категорией, важной для исследования, является уровень дохода. Взглянем на распределение значений в колонке `total_income`:

In [35]:
data['total_income'].describe().apply("{:.0f}".format)

count      21454
mean      165485
std        98335
min        20667
25%       107485
50%       143334
75%       197884
max      2265604
Name: total_income, dtype: object

75% клиентов имеют доход ниже 200 тысяч в месяц, но при этом самые богатые клиенты зарабывают на порядок больше.

Линейное разбиение здесь выглядит нецелесообразным, разумно было бы воспользоваться теми же пороговыми значениями, что и в выводе *describe()*. Делать это вручную мы, конечно, не будем, т.к. существуют *cut()* и *qcut()*, делающие это в одну строку:

In [36]:
# Автоматическая категоризация на 4 группы.
data['income_group'] = pd.qcut (data['total_income'], q=4, labels=['низкий','средний','высокий','очень высокий'])

Это полностью аналогично ручной категоризации с применением через *apply()* функции такого вида:

In [37]:
def income_group (total_income):
    try:
        if total_income <= 107623:
            return 'низкий'
        elif total_income <= 142594:
            return 'средний'
        elif total_income <= 195820:
            return 'высокий'
        else:
            return 'очень высокий'
    except:
        return '<данные повреждены>'

но намного проще и надежнее.

In [38]:
data['income_group'].value_counts()

низкий           5364
очень высокий    5364
средний          5363
высокий          5363
Name: income_group, dtype: int64

**Вывод**

В разделе "лемматизация" мы ввели в таблицу категорию базовой цели кредитования (`purpose_group`).
Для нашего исследования мы также ввели бинарную категорию наличия детей (`has_children`), а также категоризацию по четырем уровням дохода (`income_group`), стараясь получить примерно равное количество наблюдений для каждой категории (принцип квантилей).

Кроме того, на этапе предобработки данных была введена категоризация по возрастным группам (`age_group`), которая может быть полезна при дальнейших исследованиях.

В датасете уже имеются словари `education_id`:`education` и `family_status_id`:`family_status`. В корректности `education_id` мы убедились на этапе обработки пустых значений, можно проверить и `family_status_id`:

In [39]:
data.groupby('family_status_id')['family_status'].value_counts()

family_status_id  family_status        
0                 женат / замужем          12339
1                 гражданский брак          4151
2                 вдовец / вдова             959
3                 в разводе                 1195
4                 Не женат / не замужем     2810
Name: family_status, dtype: int64

Значения `family_status_id` и `family_status` полностью соответствуют.

## Вопросы исследования <a id="analysis"></a>

### Есть ли зависимость между наличием детей и возвратом кредита в срок? <a id="children"></a>

In [40]:
# Составим сводную таблицу задолженности по количеству детей у клиента:
num_children_pivot = data.pivot_table (index=['children'], values='debt', aggfunc=('count','sum','mean'))
num_children_pivot[["count", "sum", "mean"]]

,count,sum,mean
children,,,
0,14091.0,1063.0,0.075438
1,4855.0,445.0,0.091658
2,2128.0,202.0,0.094925
3,330.0,27.0,0.081818
4,41.0,4.0,0.097561
5,9.0,0.0,0.000000


Прямой зависимости задолженности от количества детей не наблюдается: клиенты с 1, 2 и 4 детьми выплачивают кредиты заметно хуже (9,2-9,8% просрочки), чем бездетные (7.5%), но клиенты с 3 детьми ближе к бездетным по этому показателю (8,2%), а у клиентов с 5 детьми задолженностей нет вовсе.

Но отметим, что чем больше число детей, тем менее репрезентативной выглядит выборка - слишком мало число наблюдений.

In [41]:
# Составим сводную таблицу задолженности по наличию детей у клиента:
has_children_pivot = data.pivot_table (index=['has_children'], values='debt', aggfunc=('count','sum','mean'))
has_children_pivot[["count", "sum", "mean"]]

,count,sum,mean
has_children,,,
0,14091.0,1063.0,0.075438
1,7363.0,678.0,0.092082


Тем не менее, бездетные в целом платят более исправно - лишь 7,5% бездетных клиентов имеют задолженность против 9,2% у клиентов с детьми.

**Вывод**

Прямой зависимости между количеством детей и задолженностью не обнаружено, но в целом бездетные действительно имеют меньше долгов по кредитам, что вполне объяснимо: в общечеловеческой системе ценностей нужды ребенка, реальные или мнимые, оказываются важнее задолженности перед банком.

### Есть ли зависимость между семейным положением и возвратом кредита в срок? <a id="fam_status"></a>

In [42]:
# Составим сводную таблицу задолженности по семейному положению клиента:
family_status_pivot = data.pivot_table(index=['family_status'], values='debt', aggfunc=('count','sum','mean'))
# Для наглядности отсортируем ее по средней доле задолженностей:
family_status_pivot = family_status_pivot.sort_values(by='mean')
family_status_pivot[["count", "sum", "mean"]]

,count,sum,mean
family_status,,,
вдовец / вдова,959.0,63.0,0.065693
в разводе,1195.0,85.0,0.071130
женат / замужем,12339.0,931.0,0.075452
гражданский брак,4151.0,388.0,0.093471
Не женат / не замужем,2810.0,274.0,0.097509


**Вывод**

У людей в браке и бывших в браке задолженностей по кредитам заметно меньше (6,6-7,5%), чем у холостых или просто сожительствующих (9,3-9,8%). Можно заключить, что опыт семейной жизни учит более ответственному отношению к финансам.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок? <a id="income"></a>

In [43]:
# Составим сводную таблицу задолженности по категории дохода клиента:
income_group_pivot = data.pivot_table (index=['income_group'], values='debt', aggfunc=('count','sum','mean'))
income_group_pivot[["count", "sum", "mean"]]

,count,sum,mean
income_group,,,
низкий,5364.0,427.0,0.079605
средний,5363.0,484.0,0.090248
высокий,5363.0,456.0,0.085027
очень высокий,5364.0,374.0,0.069724


**Вывод**

Клиенты с очень высоким уровнем заработка (верхние 25%) имеют наименьшую задолженность (7% просрочек). Однако самые бедные клиенты (нижние 25%) оказываются более дисциплинированными, чем "средний класс": 8% невозвратов против 8,5-8,8%.

В целом прямой зависимости между доходом и долгами не прослеживается. Тем не менее, можно сделать очевидные наблюдения: очень богатые клиенты имеют меньше долгов, чем все остальные, а люди с ограниченным доходом волей-неволей вынуждены блюсти финансовую дисциплину.

### Как разные цели кредита влияют на его возврат в срок? <a id="purpose"></a>

In [44]:
# Составим сводную таблицу задолженности по основной цели кредита:
purpose_group_pivot = data.pivot_table(index=['purpose_group'], values = 'debt', aggfunc=('count','sum','mean'))

# Для наглядности отсортируем ее по средней доле задолженностей:
purpose_group_pivot = purpose_group_pivot.sort_values(by='mean')
purpose_group_pivot[["count", "sum", "mean"]]

,count,sum,mean
purpose_group,,,
недвижимость,10811.0,782.0,0.072334
свадьба,2324.0,186.0,0.080034
образование,4013.0,370.0,0.092200
автомобиль,4306.0,403.0,0.093590


**Вывод**

Зависимость между целью получения кредита и процентом невозврата действительно прослеживается: кредиты на покупку недвижимости показывают наименьшую долю просрочек (около 7,2%), следом идут кредиты на свадьбу (около 8%). Клиенты, берущие кредиты на образование и покупку авто демонстрируют несколько меньшую старательность в воврате кредита, процент задолженности в этих двух категориях одного порядка (9,2-9,4%).

## Общий вывод <a id="conclusion"></a>

Мы обработали и проанализировали данные о платежеспособности клиентов и можем сделать следующие выводы:

    
- Семейное положение действительно влияет на погашение кредита в срок. Клиенты в браке чаще платят в срок, чем вне брака, а разведенные и овдовевшие еще чаще: самыми ответственными оказались вдовцы - в среднем лишь 6,6% просрочек, самыми ненадежными неженатые/незамужние - 9,8% из них не платят вовремя. 

    
- Бездетные клиенты имеют меньше задолженностей, чем клиенты с детьми; разное количество детей не коррелирует с фактом наличия задолженности. Так, ни один клиент с 5 детьми не должен банку, а у клиентов с 4 детьми долгов больше всего - 9,8%, но таких людей слишком мало, чтобы это было статистически значимым. В целом лишь 7,5% бездетных клиентов не гасят кредит в срок против 9,2% у клиентов с детьми.

    
- Клиенты с доходом выше 200 тыс. в месяц реже прочих оказываются в должниках - таковых среди них всего 7%. За ними следуют клиенты с самой низкой категорией дохода (20-100 тыс. в месяц), а вот люди со средним доходом гасят кредит неохотно. Клиенты с доходом 107-143 тыс. оказались наименее ответственными - 9% задолженностей.

    
- Кредиты на жилье и свадьбы выплачивают заметно старательней, чем на образование и покупку авто. Самыми ответственными оказались клиенты, берущие кредит на покупку недвижимости - должников лишь 7,2%, а худшими клиентами - взявшие кредит ради автомобиля, - 9,4% таких кредитов просрочено.

Идеальный клиент банка по этим параметрам - бездетный, состоящий или ранее состоявший в браке, с доходом выше 200 тыс. в месяц, ищущий кредит на покупку недвижимости.

In [45]:
# Продемонстрируем правильность нашего вывода: сравним среднего и нашего "идеального" клиента:
ideal_data = data[(data['has_children']==0) & (data['family_status_id'].isin([0,2,3])) &
                  (data['income_group']=='очень высокий') & (data['purpose_group']=='недвижимость')]
print ('Средняя доля задолженностей по всем клиентам: {:.1%}'.format (data['debt'].mean()))
print ('Средняя доля задолженностей среди идеальных клиентов: {:.1%}'.format (ideal_data['debt'].mean()))

Средняя доля задолженностей по всем клиентам: 8.1%
Средняя доля задолженностей среди идеальных клиентов: 6.0%


### Рекомендации по заполнению данных:

- Текущий метод заполнения сведений об образовании приводит к разнобою и дубликатам. Рекомендуется выбирать категорию из списка.
- Данные о трудовом стаже неясны. Они не используются в данном исследовании, но неплохо было бы получить разъяснения от составителей.
- Было бы желательно ввести уникальный идентификатор клиента - без этого неясно, как трактовать идентичные строки.